### API for coordinates

In [8]:
import requests
import numpy as np
import pandas as pd
import json

In [13]:
address = input("Enter the  address: ") 

Enter the  address: Sint-Veerleplein 11 9000 Gent


In [14]:
def get_coordinates(address: str):
    req = requests.get(f"https://loc.geopunt.be/v4/Location?q={address}").json()
    info = {'address' : address, 
                'x_value' : req['LocationResult'][0]['Location']['X_Lambert72'],
                'y_value' : req['LocationResult'][0]['Location']['Y_Lambert72'],
                'street' : req['LocationResult'][0]['Thoroughfarename'],
                'house_number' : req['LocationResult'][0]['Housenumber'], 
                'postcode': req['LocationResult'][0]['Zipcode'], 
                'municipality' : req['LocationResult'][0]['Municipality']}
    
    detail = requests.get("https://api.basisregisters.vlaanderen.be/v1/adresmatch", 
                          params={"postcode": info['postcode'], 
                                  "straatnaam": info['street'],
                                  "huisnummer": info['house_number']}).json()
    building = requests.get(detail['adresMatches'][0]['adresseerbareObjecten'][0]['detail']).json()
    build = requests.get(building['gebouw']['detail']).json()
    info['polygon'] = [build['geometriePolygoon']['polygon']]
    return info['polygon'][0]['coordinates'][0] 

In [15]:
adresss_coordination = get_coordinates(address)
adresss_coordination

[[104594.1234998703, 194294.18976890296],
 [104588.81265186518, 194296.49325690418],
 [104585.20394786447, 194287.56500089914],
 [104587.39946786314, 194287.0782168992],
 [104586.86762785912, 194285.23168889806],
 [104587.48791586608, 194283.7977048941],
 [104587.7096118629, 194283.69031289592],
 [104587.93233186007, 194283.59047289565],
 [104588.15684386343, 194283.5057368949],
 [104588.38429986686, 194283.44378489628],
 [104588.6154678613, 194283.41216889396],
 [104588.74692386389, 194283.40999289602],
 [104588.87940386683, 194283.41844089702],
 [104589.01239586622, 194283.43610489368],
 [104589.14551586658, 194283.4617048949],
 [104589.27812386304, 194283.49383289367],
 [104589.47626786679, 194283.55309689417],
 [104589.66890786588, 194283.6263128966],
 [104589.85233186185, 194283.715272896],
 [104590.02308386564, 194283.82202489674],
 [104590.17757986486, 194283.94848889485],
 [104590.31223586202, 194284.09389689565],
 [104590.4301238656, 194284.2559448965],
 [104590.53367586434, 1